In [1]:
import numpy as np
import glob
import os
import cPickle as pickle
from collections import defaultdict

import shapefile as shp
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

import matplotlib.path as mplPath

import matplotlib.lines as mlines

import shutil
import seaborn as sns

from shutil import copyfile

%matplotlib inline

In [2]:
# eppy paramters
from eppy import modeleditor
from eppy.modeleditor import IDF
# set the idd file
iddfile = 'Energy+.idd'
IDF.setiddname(iddfile)

get building occupancies

In [3]:
with open('../PickledData/UBEMdata/buildings.pickle', 'rb') as f:
    oldVals = pickle.load(f)

In [4]:
with open('../PickledData/UBEMdata/SDLBuildings.pickle', 'rb') as f:
    SDLBuildings = pickle.load(f)

In [5]:
plts = ['light', 'eqp', 'dhw', 'het']

In [6]:
schedUsetoSDLUse = {}
schedUsetoSDLUse['RetS'] = 'RETAIL'
schedUsetoSDLUse['Mall'] = 'MALL'
schedUsetoSDLUse['Gara'] = 'GARAGE'
schedUsetoSDLUse['OffM'] = 'OFFICE'
schedUsetoSDLUse['OffL'] = 'OFFICE'
schedUsetoSDLUse['ReSd'] = 'RESTRNT'
schedUsetoSDLUse['AppM'] = 'RESIDENT'
schedUsetoSDLUse['HotL'] = 'HOTEL'
schedUsetoSDLUse['HotS'] = 'HOTEL'

In [7]:
problemBees = []
for j,b in enumerate(oldVals.keys()):
    usesSet = set([oldVals[b]['zones'][z]['use'] for z in oldVals[b]['zones']])
    for use in usesSet:
        for j,p in enumerate(plts):
            kPrime = p+'Schedules'
            try:
                k = [k for k in oldVals[b][kPrime].keys() if use in k][0]
            except:
                pass
        try:
            SDLBuildings[b][schedUsetoSDLUse[use]]['occupancy24']
        except:
            problemBees.append(b)
print problemBees

['0501258000', '0501395000', '0501370000', '0501312000', '0501301000', '0501315000', '0501390000']


In [8]:
# delete the problem buildings
for bID in set(problemBees):
    del oldVals[bID]

In [9]:
multiUses = []
single = 0
singleUses = []
for j,b in enumerate(oldVals.keys()):
    usesSet = set([oldVals[b]['zones'][z]['use'] for z in oldVals[b]['zones']])
    if len(usesSet)==1:
        single+=1
        singleUses.append( oldVals[b]['zones'][z]['use'] )
        
    else:
        multiUses.append([use for use in usesSet])
            
print 'There are', single, 'single use buildings'

There are 195 single use buildings


In [10]:
# for zU in set(singleUses):
#     print 'There are', len([x for x in singleUses if x==zU]), zU, 'buildings'
# #print multiUses
# unique_data = [list(x) for x in set(tuple(x) for x in multiUses)]
# #print unique_data
# for u in unique_data:
#     print u
#     print len([x for x in multiUses if x==u])

In [11]:
def scenario_1(oldSchedule, oldOccupancy, newOccupancy):
    newSchedule = np.zeros(np.shape(oldSchedule))
    for oi in range(len(newSchedule)):
        if oldOccupancy[oi]>0:
            if np.max(newOccupancy)>0:
                newSchedule[oi] = np.min(oldSchedule) + (oldSchedule[oi]-np.min(oldSchedule)) \
                *((newOccupancy[oi]/np.max(newOccupancy))/(oldOccupancy[oi]/np.max(oldOccupancy)))
            else:
                newSchedule[oi] = np.min(oldSchedule)
        else:
            newSchedule[oi] = np.min(oldSchedule)
        # if there are less people dont go above reference
        if oldOccupancy[oi]>=newOccupancy[oi]:
            if newSchedule[oi]>oldSchedule[oi]:
                newSchedule[oi] = oldSchedule[oi]
        # if there are more people dont go below reference
        if newOccupancy[oi]>oldOccupancy[oi]:
            if newSchedule[oi]<oldSchedule[oi]:
                newSchedule[oi] = oldSchedule[oi]
            # new can only scale above ref in proportion
            if newSchedule[oi]>oldSchedule[oi]*(newOccupancy[oi]/oldOccupancy[oi]):
                newSchedule[oi] = oldSchedule[oi]*(newOccupancy[oi]/oldOccupancy[oi])
            # limit the load so that it cannot be hogher unless people higher
            if np.max(oldOccupancy)>=np.max(newOccupancy):
                if newSchedule[oi]>np.max(oldSchedule):
                    newSchedule[oi] = np.max(oldSchedule)
        if np.isnan(newSchedule[oi]):
            print 'uh oh'
    return newSchedule

def scenario_2(oldSchedule, oldOccupancy, newOccupancy):
    newSchedule = np.zeros(np.shape(oldSchedule))
    for oi in range(len(newSchedule)):
        if oldOccupancy[oi]>0:
            if np.max(newOccupancy)>0:
                newSchedule[oi] = np.min(oldSchedule)+(oldSchedule[oi]-np.min(oldSchedule)) \
                *(newOccupancy[oi]/oldOccupancy[oi])
            else:
                newSchedule[oi] = np.min(oldSchedule)
            if np.max(oldOccupancy)>=np.max(newOccupancy):
                if newSchedule[oi]>np.max(oldSchedule):
                    newSchedule[oi] = np.max(oldSchedule)
        else:
            newSchedule[oi] = np.min(oldSchedule)
        if np.isnan(newSchedule[oi]):
            print 'uh oh'
    return newSchedule

In [12]:
# for each building and use change the schedules
predictedSce1 = {}
predictedSce2 = {}
for j,b in enumerate(oldVals.keys()):
    hrs = np.arange(0,24)
    usesSet = set([oldVals[b]['zones'][z]['use'] for z in oldVals[b]['zones']])
    predictedSce1[b] = {}
    predictedSce2[b] = {}
    for use in usesSet:
        predictedSce1[b][use] = {}
        predictedSce2[b][use] = {}
        for jay,p in enumerate(plts):
            kPrime = p+'Schedules'
            try:
                k = [k for k in oldVals[b][kPrime].keys() if use in k][0]
                hourly = oldVals[b][p][oldVals[b][kPrime][k]]
            except:
                pass
        newOccUse = SDLBuildings[b][schedUsetoSDLUse[use]]['occupancy24']
        
        predictedSce1[b][use]['occupancy'] = newOccUse
        predictedSce2[b][use]['occupancy'] = newOccUse
        total_occ = np.zeros((24))
        for z in oldVals[b]['zones'].values():
            if use in z['people']:
                total_occ += np.asarray( oldVals[b]['occupancy'][oldVals[b]['peopleSchedules'][z['people']]] ) \
                            *z['area']*z['PCA']
        oldOccUse = np.round(total_occ)
        
        for jay,p in enumerate(plts):
            kPrime = p+'Schedules'
            try:
                k = [k for k in oldVals[b][kPrime].keys() if use in k][0]
                oldSchedule = oldVals[b][p][oldVals[b][kPrime][k]]
                if p in ['light', 'eqp', 'dhw']:
                    minVal = min(oldSchedule)
                    
                    newSchedule1 = scenario_1(oldSchedule, oldOccUse, newOccUse)

                    newSchedule2 = scenario_2(oldSchedule, oldOccUse, newOccUse)

                    predictedSce1[b][use][p] = newSchedule1
                    predictedSce2[b][use][p] = newSchedule2             

            except:
                pass

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [13]:
fileList = glob.glob('../Full_idf_files/*_0.idf')
print len(fileList)

325


In [14]:
# delete the files from list with problems
fileList = [file1 for file1 in fileList if not any(substring in file1 for substring in set(problemBees))]

In [15]:
print len(fileList)

318


In [16]:
# remove any old files
listFile=glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*.idf')
print len(listFile)
for file1 in listFile:
    os.remove(file1)

954


In [17]:
for f in fileList:
    newName = '../Idf_generated_files/All_buildings_standard_efficiency/'\
    +f.split('Full_idf_files/')[1].split('_')[0]+'_control.idf'
    copyfile(f, newName)
    newName = '../Idf_generated_files/All_buildings_standard_efficiency/'\
    +f.split('Full_idf_files/')[1].split('_')[0]+'_experiment1.idf'
    copyfile(f, newName)
    newName = '../Idf_generated_files/All_buildings_standard_efficiency/'\
    +f.split('Full_idf_files/')[1].split('_')[0]+'_experiment2.idf'
    copyfile(f, newName)

In [18]:
print len(glob.glob('../Full_idf_files/*_0.idf'))
print len(glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_control.idf'))
print len(glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_experiment1.idf'))
print len(glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_experiment2.idf'))

325
318
318
318


In [19]:
# This information is contained within the header of the Boston EPW file
# typical winter week Jan 27 to Feb 2
# typical spring week Mar 29 to Apr 4
# typical summer week Jul 27 to Aug 2
# typcial autumn week Oct 20 to Oct 26
# take the middle day in the typical week
beginMonth = [1,4,7,10]
endMonth = [1,4,7,10]
beginDayMonth = [30,1,30,23]

In [20]:
def change_runtime(idfContents, beginMonth, endMonth, beginDayMonth):
    runList = idfContents.idfobjects['RUNPERIOD'] 
    for j in range(3):
        idfContents.copyidfobject(runList[0])   
    runList = idf1.idfobjects['RUNPERIOD'] 
    for k,e in enumerate(runList):
        e.Name = e.Name+'_'+str(k)
        e.Begin_Month = beginMonth[k]
        e.Begin_Day_of_Month = beginDayMonth[k]
        e.End_Month = endMonth[k]
        e.End_Day_of_Month = beginDayMonth[k]
        e.Day_of_Week_for_Start_Day = 'Monday'
    return idfContents

In [21]:
# change runtime
fileList2 = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_control.idf')
for f in fileList2:
    idf1 = IDF(f)
    idf1 = change_runtime(idf1, beginMonth, endMonth, beginDayMonth)
    idf1.save()
fileList2 = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_experiment1.idf')
for f in fileList2:
    idf1 = IDF(f)
    idf1 = change_runtime(idf1, beginMonth, endMonth, beginDayMonth)
    idf1.save()
fileList2 = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_experiment2.idf')
for f in fileList2:
    idf1 = IDF(f)
    idf1 = change_runtime(idf1, beginMonth, endMonth, beginDayMonth)
    idf1.save()

In [22]:
print 'runperiod changed'

runperiod changed


In [23]:
def get_schedule_name_and_value(idf1, searchString):
    schedules = idf1.idfobjects['SCHEDULE:YEAR']
    sYToD = {}
    for s in schedules:
        if searchString in s.Name:
            sYToD[s.Name] = s.ScheduleWeek_Name_1
    
    schedules = idf1.idfobjects['SCHEDULE:WEEK:DAILY']
    for sh in sYToD:
        name = sYToD[sh]
        for s in schedules:
            if name == s.Name:
                sYToD[sh] = s.Monday_ScheduleDay_Name
    
    return sYToD

In [24]:
fileList2 = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_experiment1.idf')
for f in fileList2:
    idf1 = IDF(f)
    buildingID = f.split('standard_efficiency/')[1][:-4].split('_')[0]

    zoneNames = [ob.Name for ob in idf1.idfobjects['ZONE']]
    zoneObjects = idf1.idfobjects['ZONE']
    people = idf1.idfobjects['PEOPLE']
    lights = idf1.idfobjects['LIGHTS']
    equipment = idf1.idfobjects['ELECTRICEQUIPMENT']
    heat = idf1.idfobjects['ZONEHVAC:IDEALLOADSAIRSYSTEM']
    DHW = idf1.idfobjects['WATERUSE:EQUIPMENT']
    zoneUses = []
    zoneSchedules = []

    for j,z in enumerate(zoneNames):
        testing = people[j].Number_of_People_Schedule_Name.split('B_')[1][0:4]
        if testing not in zoneUses:
            zoneUses.append(testing)
            zoneSchedules.append(people[j].Number_of_People_Schedule_Name)
    ################## sort out to change area per person

    sYToD = get_schedule_name_and_value(idf1, 'Occ')

    for jjj,zU in enumerate(zoneUses):
        # each zone with the same use is assigned the same occupancy
        newS = predictedSce1[buildingID][zU]['occupancy']/max( oldVals[buildingID][zU] )

        if any(newS>1):
            multiplier = np.max(newS)
            # reset newS
            newS = newS/multiplier
            #loop through zones
            for zoneIterator, peepShed in enumerate(people):
                if peepShed.Number_of_People_Schedule_Name.split('B_')[1][0:4]==zU:
                    # change the Watts per zone floor area
                    people[zoneIterator].People_per_Zone_Floor_Area = \
                    people[zoneIterator].People_per_Zone_Floor_Area*multiplier
        
        #get schedule Name
        schName = zoneSchedules[jjj]
        schName = sYToD[schName]
        #print schName
        # get that sched
        schedules = idf1.idfobjects['SCHEDULE:DAY:HOURLY']
        for s in schedules:
            if s.Name==schName:
                # change the schedule
                for kkk in range(24):
                    s['obj'][3+kkk]=round(newS[kkk],3)

    ################# LIGHT
    
    sYToDL = get_schedule_name_and_value(idf1, 'Lgt')

    for jjj,zU in enumerate(zoneUses):
        newS = predictedSce1[buildingID][zU]['light']
        if any(newS>1):
            multiplier = np.max(newS)
            # reset newS
            newS = newS/multiplier
            #loop through zones
            for zoneIterator, peepShed in enumerate(people):
                if peepShed.Number_of_People_Schedule_Name.split('B_')[1][0:4]==zU:
                    # change the Watts per zone floor area
                    lights[zoneIterator].Watts_per_Zone_Floor_Area = \
                    lights[zoneIterator].Watts_per_Zone_Floor_Area*multiplier
        
        #get schedule Name
        schName = [x for x in sYToDL.keys() if zU in x][0]
        schName = sYToDL[schName]
        # get that sched
        schedules = idf1.idfobjects['SCHEDULE:DAY:HOURLY']
        for s in schedules:
            if s.Name==schName:
                #print s
                # change the schedule
                for kkk in range(24):
                    s['obj'][3+kkk]=round(newS[kkk],3)
                #print s

    ################## EQUIPMENT
    
    sYToDEqp = get_schedule_name_and_value(idf1, 'Eqp')

    for jjj,zU in enumerate(zoneUses):
        #get old and new eqp
        newS = predictedSce1[buildingID][zU]['eqp']
        if any(newS>1):
            multiplier = np.max(newS)
            # reset newS
            newS = newS/multiplier
            #loop through zones
            for zoneIterator, peepShed in enumerate(people):
                if peepShed.Number_of_People_Schedule_Name.split('B_')[1][0:4]==zU:
                    # change the Watts per zone floor area
                    equipment[zoneIterator].Watts_per_Zone_Floor_Area = \
                    equipment[zoneIterator].Watts_per_Zone_Floor_Area*multiplier
                    
        #get schedule Name
        schName = [x for x in sYToDEqp.keys() if zU in x][0]
        schName = sYToDEqp[schName]
        # get that sched
        schedules = idf1.idfobjects['SCHEDULE:DAY:HOURLY']
        for s in schedules:
            if s.Name==schName:
                # change the schedule
                for kkk in range(24):
                    s['obj'][3+kkk]=round(newS[kkk],3)

    ######################### DHW
    
    sYToDDhw = get_schedule_name_and_value(idf1, 'DHW')

    for jjj,zU in enumerate(zoneUses):
        #get old and new dhw
        try:
            newS = predictedSce1[buildingID][zU]['dhw']
            if any(newS>1):
                multiplier = np.max(newS)
                # reset newS
                newS = newS/multiplier # now loop through zones
                for zoneIterator, peepShed in enumerate(people):
                    if peepShed.Number_of_People_Schedule_Name.split('B_')[1][0:4]==zU:
                        # change the Watts per zone floor area
                        DHW[zoneIterator].Peak_Flow_Rate = \
                        DHW[zoneIterator].Peak_Flow_Rate*multiplier    
            
            #get schedule Name
            schName = [x for x in sYToDDhw.keys() if zU in x][0]
            schName = sYToDDhw[schName]
            # get that sched
            schedules = idf1.idfobjects['SCHEDULE:DAY:HOURLY']
            for s in schedules:
                if s.Name==schName:
                    # change the schedule
                    for kkk in range(24):
                        s['obj'][3+kkk]=round(newS[kkk],3)
        except:
            pass
    idf1.save()


In [25]:
fileList2 = glob.glob('../Idf_generated_files/All_buildings_standard_efficiency/*_experiment2.idf')
for f in fileList2:
    idf1 = IDF(f)
    buildingID = f.split('standard_efficiency/')[1][:-4].split('_')[0]

    zoneNames = [ob.Name for ob in idf1.idfobjects['ZONE']]
    zoneObjects = idf1.idfobjects['ZONE']
    people = idf1.idfobjects['PEOPLE']
    lights = idf1.idfobjects['LIGHTS']
    equipment = idf1.idfobjects['ELECTRICEQUIPMENT']
    heat = idf1.idfobjects['ZONEHVAC:IDEALLOADSAIRSYSTEM']
    DHW = idf1.idfobjects['WATERUSE:EQUIPMENT']
    zoneUses = []
    zoneSchedules = []

    for j,z in enumerate(zoneNames):
        testing = people[j].Number_of_People_Schedule_Name.split('B_')[1][0:4]
        if testing not in zoneUses:
            zoneUses.append(testing)
            zoneSchedules.append(people[j].Number_of_People_Schedule_Name)
    ################## sort out to change area per person

    sYToD = get_schedule_name_and_value(idf1, 'Occ')

    for jjj,zU in enumerate(zoneUses):
        #get old and new occupancy
        newS = predictedSce2[buildingID][zU]['occupancy']/max( oldVals[buildingID][zU] )
        if any(newS>1):
            multiplier = np.max(newS)
            # reset newS
            newS = newS/multiplier
            #loop through zones
            for zoneIterator, peepShed in enumerate(people):
                if peepShed.Number_of_People_Schedule_Name.split('B_')[1][0:4]==zU:
                    # change the Watts per zone floor area
                    people[zoneIterator].People_per_Zone_Floor_Area = \
                    people[zoneIterator].People_per_Zone_Floor_Area*multiplier
        #get schedule Name
        schName = zoneSchedules[jjj]
        schName = sYToD[schName]
        #print schName
        # get that sched
        schedules = idf1.idfobjects['SCHEDULE:DAY:HOURLY']
        for s in schedules:
            if s.Name==schName:
                # change the schedule
                #print s['obj'][3:]
                for kkk in range(24):
                    s['obj'][3+kkk]=round(newS[kkk],3)
                #print s['obj'][3:]

    ################# LIGHT

    sYToDL = get_schedule_name_and_value(idf1, 'Lgt')

    for jjj,zU in enumerate(zoneUses):
        newS = predictedSce2[buildingID][zU]['light']
        if any(newS>1):
            multiplier = np.max(newS)
            # reset newS
            newS = newS/multiplier
            #loop through zones
            for zoneIterator, peepShed in enumerate(people):
                if peepShed.Number_of_People_Schedule_Name.split('B_')[1][0:4]==zU:
                    # change the Watts per zone floor area
                    lights[zoneIterator].Watts_per_Zone_Floor_Area = \
                    lights[zoneIterator].Watts_per_Zone_Floor_Area*multiplier
        
        
        #get schedule Name
        schName = [x for x in sYToDL.keys() if zU in x][0]
        schName = sYToDL[schName]
        # get that sched
        schedules = idf1.idfobjects['SCHEDULE:DAY:HOURLY']
        for s in schedules:
            if s.Name==schName:
                #print s
                # change the schedule
                for kkk in range(24):
                    s['obj'][3+kkk]=round(newS[kkk],3)
                #print s

    ################## EQUIPMENT

    sYToDEqp = get_schedule_name_and_value(idf1, 'Eqp')

    for jjj,zU in enumerate(zoneUses):
        #get old and new eqp
        newS = predictedSce2[buildingID][zU]['eqp']
        if any(newS>1):
            multiplier = np.max(newS)
            # reset newS
            newS = newS/multiplier
            #loop through zones
            for zoneIterator, peepShed in enumerate(people):
                if peepShed.Number_of_People_Schedule_Name.split('B_')[1][0:4]==zU:
                    # change the Watts per zone floor area
                    equipment[zoneIterator].Watts_per_Zone_Floor_Area = \
                    equipment[zoneIterator].Watts_per_Zone_Floor_Area*multiplier
        #print newS
        schName = [x for x in sYToDEqp.keys() if zU in x][0]
        schName = sYToDEqp[schName]
        # get that sched
        schedules = idf1.idfobjects['SCHEDULE:DAY:HOURLY']
        for s in schedules:
            if s.Name==schName:
                # change the schedule
                for kkk in range(24):
                    s['obj'][3+kkk]=round(newS[kkk],3)


    ######################### DHW

    sYToDDhw = get_schedule_name_and_value(idf1, 'DHW')

    for jjj,zU in enumerate(zoneUses):
        #get old and new dhw
        try:
            newS = predictedSce2[buildingID][zU]['dhw']
            if any(newS>1):
                multiplier = np.max(newS)
                # reset newS
                newS = newS/multiplier # now loop through zones
                for zoneIterator, peepShed in enumerate(people):
                    if peepShed.Number_of_People_Schedule_Name.split('B_')[1][0:4]==zU:
                        # change the Watts per zone floor area
                        DHW[zoneIterator].Peak_Flow_Rate = \
                        DHW[zoneIterator].Peak_Flow_Rate*multiplier 
                        
            #get schedule Name
            schName = [x for x in sYToDDhw.keys() if zU in x][0]
            schName = sYToDDhw[schName]
            # get that sched
            schedules = idf1.idfobjects['SCHEDULE:DAY:HOURLY']
            for s in schedules:
                if s.Name==schName:
                    # change the schedule
                    for kkk in range(24):
                        s['obj'][3+kkk]=round(newS[kkk],3)
        except:
            pass

    idf1.save()


In [27]:
print 'done'

done
